In [2]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def firsts(list):
    answer=[]
    for pair in list:
        answer=answer+[pair[0]]
    return answer

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jLeoPowerConstantTerm(m,power,multiplier):
    trun=jLeo(power,m,multiplier)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def last_element(listo):return listo[len(listo)-1]
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay")

okay


In [ ]:
#output snipped; I interrupted this after k = 94.
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [1..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst= jpowerConstantTerm(m,power)
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-nszfunction(power) #Switched notations! nszfunction(n) identically = broadhurst(n) 
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
#below was originally done in a separate cell
#import pickle
#wfile = open('/Users/barrybrent/26oct22no1.txt','wb')
#t = pickle.dumps(str(polydata))
#wfile.write(t)
#wfile.close()
---------------------------------------------------------------------------------------------------
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [95..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst=QQ(jpowerConstantTerm(m,power))
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-broadhurst(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
import pickle
wfile = open('/Users/barrybrent/26oct22no2.txt','wb')
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
print("tests:")
print(tests)

In [2]:
a=[1,2,3];b=[4,5,6];c=a+b;print(c)

[1, 2, 3, 4, 5, 6]


In [6]:
import pickle
rsfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)

rtfile = open('/Users/barrybrent/26oct22no2.txt','rb')
rt=pickle.load(rtfile)
rtfile.close()
t=stripQuotationMarks(rt)

u=s+t

wufile = open('/Users/barrybrent/5dec22no1.txt','wb')
u = pickle.dumps(str(u))
wufile.write(u)

1968604

In [4]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print(len(s))

100


In [9]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print(len(s))

100


In [3]:
#  conjecture 5.1
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
p=2
power=s[1][0] # = 2
poly=D(s[1][1]) # poly(m) = cst term of j_m^power
nf=QQ(numericalfactor(poly))
polyd=expand(poly/nf)
for j in [0..200]:
    m=2^j
    polydm=evaluate(polyd,m)
    o=order(p,polydm)
    core=polydm/p^o
    mcp=mod(core,p)
    test=o-(2*j+4)
    tupl=(p,j,power,mcp,test)
    data=data+[test]
    print(tupl)
print(data)

(2, 0, 2, 1, -4)
(2, 1, 2, 1, 6)
(2, 2, 2, 1, 0)
(2, 3, 2, 1, 0)
(2, 4, 2, 1, 0)
(2, 5, 2, 1, 0)
(2, 6, 2, 1, 0)
(2, 7, 2, 1, 0)
(2, 8, 2, 1, 0)
(2, 9, 2, 1, 0)
(2, 10, 2, 1, 0)
(2, 11, 2, 1, 0)
(2, 12, 2, 1, 0)
(2, 13, 2, 1, 0)
(2, 14, 2, 1, 0)
(2, 15, 2, 1, 0)
(2, 16, 2, 1, 0)
(2, 17, 2, 1, 0)
(2, 18, 2, 1, 0)
(2, 19, 2, 1, 0)
(2, 20, 2, 1, 0)
(2, 21, 2, 1, 0)
(2, 22, 2, 1, 0)
(2, 23, 2, 1, 0)
(2, 24, 2, 1, 0)
(2, 25, 2, 1, 0)
(2, 26, 2, 1, 0)
(2, 27, 2, 1, 0)
(2, 28, 2, 1, 0)
(2, 29, 2, 1, 0)
(2, 30, 2, 1, 0)
(2, 31, 2, 1, 0)
(2, 32, 2, 1, 0)
(2, 33, 2, 1, 0)
(2, 34, 2, 1, 0)
(2, 35, 2, 1, 0)
(2, 36, 2, 1, 0)
(2, 37, 2, 1, 0)
(2, 38, 2, 1, 0)
(2, 39, 2, 1, 0)
(2, 40, 2, 1, 0)
(2, 41, 2, 1, 0)
(2, 42, 2, 1, 0)
(2, 43, 2, 1, 0)
(2, 44, 2, 1, 0)
(2, 45, 2, 1, 0)
(2, 46, 2, 1, 0)
(2, 47, 2, 1, 0)
(2, 48, 2, 1, 0)
(2, 49, 2, 1, 0)
(2, 50, 2, 1, 0)
(2, 51, 2, 1, 0)
(2, 52, 2, 1, 0)
(2, 53, 2, 1, 0)
(2, 54, 2, 1, 0)
(2, 55, 2, 1, 0)
(2, 56, 2, 1, 0)
(2, 57, 2, 1, 0)
(2, 58, 2, 1, 0)
(2, 59

In [4]:
# conjecture 5.2
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
p=3
power=s[2][0] # = 3
poly=D(s[2][1]) # poly(m) = cst term of j_m^power
nf=QQ(numericalfactor(poly))
polyd=expand(poly/nf)
for j in [1..200]:
    m=3^j
    polydm=evaluate(polyd,m)
    o=order(p,polydm)
    core=polydm/p^o
    mcp=mod(core,p)
    test=o-(3*j-3)
    tupl=(p,j,power,mcp,test)
    data=data+[test]
    print(tupl)
print(data)

(3, 1, 3, 1, 0)
(3, 2, 3, 1, 0)
(3, 3, 3, 1, 0)
(3, 4, 3, 1, 0)
(3, 5, 3, 1, 0)
(3, 6, 3, 1, 0)
(3, 7, 3, 1, 0)
(3, 8, 3, 1, 0)
(3, 9, 3, 1, 0)
(3, 10, 3, 1, 0)
(3, 11, 3, 1, 0)
(3, 12, 3, 1, 0)
(3, 13, 3, 1, 0)
(3, 14, 3, 1, 0)
(3, 15, 3, 1, 0)
(3, 16, 3, 1, 0)
(3, 17, 3, 1, 0)
(3, 18, 3, 1, 0)
(3, 19, 3, 1, 0)
(3, 20, 3, 1, 0)
(3, 21, 3, 1, 0)
(3, 22, 3, 1, 0)
(3, 23, 3, 1, 0)
(3, 24, 3, 1, 0)
(3, 25, 3, 1, 0)
(3, 26, 3, 1, 0)
(3, 27, 3, 1, 0)
(3, 28, 3, 1, 0)
(3, 29, 3, 1, 0)
(3, 30, 3, 1, 0)
(3, 31, 3, 1, 0)
(3, 32, 3, 1, 0)
(3, 33, 3, 1, 0)
(3, 34, 3, 1, 0)
(3, 35, 3, 1, 0)
(3, 36, 3, 1, 0)
(3, 37, 3, 1, 0)
(3, 38, 3, 1, 0)
(3, 39, 3, 1, 0)
(3, 40, 3, 1, 0)
(3, 41, 3, 1, 0)
(3, 42, 3, 1, 0)
(3, 43, 3, 1, 0)
(3, 44, 3, 1, 0)
(3, 45, 3, 1, 0)
(3, 46, 3, 1, 0)
(3, 47, 3, 1, 0)
(3, 48, 3, 1, 0)
(3, 49, 3, 1, 0)
(3, 50, 3, 1, 0)
(3, 51, 3, 1, 0)
(3, 52, 3, 1, 0)
(3, 53, 3, 1, 0)
(3, 54, 3, 1, 0)
(3, 55, 3, 1, 0)
(3, 56, 3, 1, 0)
(3, 57, 3, 1, 0)
(3, 58, 3, 1, 0)
(3, 59, 3, 1, 0)
(3, 60

In [5]:
# conjecture 5.3
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for n in [3..100]:
    p=prime(n)
    if (p>len(s))==False:
        power=s[p-1][0] # = p.
        poly=D(s[p-1][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..200]:
            m=p^j
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            core=polydm/p^o
            mcp=mod(core,p)
            test=o-(j*p-2)
            tupl=(p,j,power,mcp,test)
            data=data+[test]
            print(tupl)
print(data)

(5, 1, 5, 3, 0)
(5, 2, 5, 3, 0)
(5, 3, 5, 3, 0)
(5, 4, 5, 3, 0)
(5, 5, 5, 3, 0)
(5, 6, 5, 3, 0)
(5, 7, 5, 3, 0)
(5, 8, 5, 3, 0)
(5, 9, 5, 3, 0)
(5, 10, 5, 3, 0)
(5, 11, 5, 3, 0)
(5, 12, 5, 3, 0)
(5, 13, 5, 3, 0)
(5, 14, 5, 3, 0)
(5, 15, 5, 3, 0)
(5, 16, 5, 3, 0)
(5, 17, 5, 3, 0)
(5, 18, 5, 3, 0)
(5, 19, 5, 3, 0)
(5, 20, 5, 3, 0)
(5, 21, 5, 3, 0)
(5, 22, 5, 3, 0)
(5, 23, 5, 3, 0)
(5, 24, 5, 3, 0)
(5, 25, 5, 3, 0)
(5, 26, 5, 3, 0)
(5, 27, 5, 3, 0)
(5, 28, 5, 3, 0)
(5, 29, 5, 3, 0)
(5, 30, 5, 3, 0)
(5, 31, 5, 3, 0)
(5, 32, 5, 3, 0)
(5, 33, 5, 3, 0)
(5, 34, 5, 3, 0)
(5, 35, 5, 3, 0)
(5, 36, 5, 3, 0)
(5, 37, 5, 3, 0)
(5, 38, 5, 3, 0)
(5, 39, 5, 3, 0)
(5, 40, 5, 3, 0)
(5, 41, 5, 3, 0)
(5, 42, 5, 3, 0)
(5, 43, 5, 3, 0)
(5, 44, 5, 3, 0)
(5, 45, 5, 3, 0)
(5, 46, 5, 3, 0)
(5, 47, 5, 3, 0)
(5, 48, 5, 3, 0)
(5, 49, 5, 3, 0)
(5, 50, 5, 3, 0)
(5, 51, 5, 3, 0)
(5, 52, 5, 3, 0)
(5, 53, 5, 3, 0)
(5, 54, 5, 3, 0)
(5, 55, 5, 3, 0)
(5, 56, 5, 3, 0)
(5, 57, 5, 3, 0)
(5, 58, 5, 3, 0)
(5, 59, 5, 3, 0)
(5, 60

(11, 176, 11, 5, 0)
(11, 177, 11, 5, 0)
(11, 178, 11, 5, 0)
(11, 179, 11, 5, 0)
(11, 180, 11, 5, 0)
(11, 181, 11, 5, 0)
(11, 182, 11, 5, 0)
(11, 183, 11, 5, 0)
(11, 184, 11, 5, 0)
(11, 185, 11, 5, 0)
(11, 186, 11, 5, 0)
(11, 187, 11, 5, 0)
(11, 188, 11, 5, 0)
(11, 189, 11, 5, 0)
(11, 190, 11, 5, 0)
(11, 191, 11, 5, 0)
(11, 192, 11, 5, 0)
(11, 193, 11, 5, 0)
(11, 194, 11, 5, 0)
(11, 195, 11, 5, 0)
(11, 196, 11, 5, 0)
(11, 197, 11, 5, 0)
(11, 198, 11, 5, 0)
(11, 199, 11, 5, 0)
(11, 200, 11, 5, 0)
(13, 1, 13, 10, 0)
(13, 2, 13, 10, 0)
(13, 3, 13, 10, 0)
(13, 4, 13, 10, 0)
(13, 5, 13, 10, 0)
(13, 6, 13, 10, 0)
(13, 7, 13, 10, 0)
(13, 8, 13, 10, 0)
(13, 9, 13, 10, 0)
(13, 10, 13, 10, 0)
(13, 11, 13, 10, 0)
(13, 12, 13, 10, 0)
(13, 13, 13, 10, 0)
(13, 14, 13, 10, 0)
(13, 15, 13, 10, 0)
(13, 16, 13, 10, 0)
(13, 17, 13, 10, 0)
(13, 18, 13, 10, 0)
(13, 19, 13, 10, 0)
(13, 20, 13, 10, 0)
(13, 21, 13, 10, 0)
(13, 22, 13, 10, 0)
(13, 23, 13, 10, 0)
(13, 24, 13, 10, 0)
(13, 25, 13, 10, 0)
(13, 26, 

(19, 51, 19, 14, 0)
(19, 52, 19, 14, 0)
(19, 53, 19, 14, 0)
(19, 54, 19, 14, 0)
(19, 55, 19, 14, 0)
(19, 56, 19, 14, 0)
(19, 57, 19, 14, 0)
(19, 58, 19, 14, 0)
(19, 59, 19, 14, 0)
(19, 60, 19, 14, 0)
(19, 61, 19, 14, 0)
(19, 62, 19, 14, 0)
(19, 63, 19, 14, 0)
(19, 64, 19, 14, 0)
(19, 65, 19, 14, 0)
(19, 66, 19, 14, 0)
(19, 67, 19, 14, 0)
(19, 68, 19, 14, 0)
(19, 69, 19, 14, 0)
(19, 70, 19, 14, 0)
(19, 71, 19, 14, 0)
(19, 72, 19, 14, 0)
(19, 73, 19, 14, 0)
(19, 74, 19, 14, 0)
(19, 75, 19, 14, 0)
(19, 76, 19, 14, 0)
(19, 77, 19, 14, 0)
(19, 78, 19, 14, 0)
(19, 79, 19, 14, 0)
(19, 80, 19, 14, 0)
(19, 81, 19, 14, 0)
(19, 82, 19, 14, 0)
(19, 83, 19, 14, 0)
(19, 84, 19, 14, 0)
(19, 85, 19, 14, 0)
(19, 86, 19, 14, 0)
(19, 87, 19, 14, 0)
(19, 88, 19, 14, 0)
(19, 89, 19, 14, 0)
(19, 90, 19, 14, 0)
(19, 91, 19, 14, 0)
(19, 92, 19, 14, 0)
(19, 93, 19, 14, 0)
(19, 94, 19, 14, 0)
(19, 95, 19, 14, 0)
(19, 96, 19, 14, 0)
(19, 97, 19, 14, 0)
(19, 98, 19, 14, 0)
(19, 99, 19, 14, 0)
(19, 100, 19, 14, 0)

(29, 66, 29, 11, 0)
(29, 67, 29, 11, 0)
(29, 68, 29, 11, 0)
(29, 69, 29, 11, 0)
(29, 70, 29, 11, 0)
(29, 71, 29, 11, 0)
(29, 72, 29, 11, 0)
(29, 73, 29, 11, 0)
(29, 74, 29, 11, 0)
(29, 75, 29, 11, 0)
(29, 76, 29, 11, 0)
(29, 77, 29, 11, 0)
(29, 78, 29, 11, 0)
(29, 79, 29, 11, 0)
(29, 80, 29, 11, 0)
(29, 81, 29, 11, 0)
(29, 82, 29, 11, 0)
(29, 83, 29, 11, 0)
(29, 84, 29, 11, 0)
(29, 85, 29, 11, 0)
(29, 86, 29, 11, 0)
(29, 87, 29, 11, 0)
(29, 88, 29, 11, 0)
(29, 89, 29, 11, 0)
(29, 90, 29, 11, 0)
(29, 91, 29, 11, 0)
(29, 92, 29, 11, 0)
(29, 93, 29, 11, 0)
(29, 94, 29, 11, 0)
(29, 95, 29, 11, 0)
(29, 96, 29, 11, 0)
(29, 97, 29, 11, 0)
(29, 98, 29, 11, 0)
(29, 99, 29, 11, 0)
(29, 100, 29, 11, 0)
(29, 101, 29, 11, 0)
(29, 102, 29, 11, 0)
(29, 103, 29, 11, 0)
(29, 104, 29, 11, 0)
(29, 105, 29, 11, 0)
(29, 106, 29, 11, 0)
(29, 107, 29, 11, 0)
(29, 108, 29, 11, 0)
(29, 109, 29, 11, 0)
(29, 110, 29, 11, 0)
(29, 111, 29, 11, 0)
(29, 112, 29, 11, 0)
(29, 113, 29, 11, 0)
(29, 114, 29, 11, 0)
(29, 

(37, 82, 37, 26, 0)
(37, 83, 37, 26, 0)
(37, 84, 37, 26, 0)
(37, 85, 37, 26, 0)
(37, 86, 37, 26, 0)
(37, 87, 37, 26, 0)
(37, 88, 37, 26, 0)
(37, 89, 37, 26, 0)
(37, 90, 37, 26, 0)
(37, 91, 37, 26, 0)
(37, 92, 37, 26, 0)
(37, 93, 37, 26, 0)
(37, 94, 37, 26, 0)
(37, 95, 37, 26, 0)
(37, 96, 37, 26, 0)
(37, 97, 37, 26, 0)
(37, 98, 37, 26, 0)
(37, 99, 37, 26, 0)
(37, 100, 37, 26, 0)
(37, 101, 37, 26, 0)
(37, 102, 37, 26, 0)
(37, 103, 37, 26, 0)
(37, 104, 37, 26, 0)
(37, 105, 37, 26, 0)
(37, 106, 37, 26, 0)
(37, 107, 37, 26, 0)
(37, 108, 37, 26, 0)
(37, 109, 37, 26, 0)
(37, 110, 37, 26, 0)
(37, 111, 37, 26, 0)
(37, 112, 37, 26, 0)
(37, 113, 37, 26, 0)
(37, 114, 37, 26, 0)
(37, 115, 37, 26, 0)
(37, 116, 37, 26, 0)
(37, 117, 37, 26, 0)
(37, 118, 37, 26, 0)
(37, 119, 37, 26, 0)
(37, 120, 37, 26, 0)
(37, 121, 37, 26, 0)
(37, 122, 37, 26, 0)
(37, 123, 37, 26, 0)
(37, 124, 37, 26, 0)
(37, 125, 37, 26, 0)
(37, 126, 37, 26, 0)
(37, 127, 37, 26, 0)
(37, 128, 37, 26, 0)
(37, 129, 37, 26, 0)
(37, 130, 

...

(53, 96, 53, 19, 0)
(53, 97, 53, 19, 0)
(53, 98, 53, 19, 0)
(53, 99, 53, 19, 0)
(53, 100, 53, 19, 0)
(53, 101, 53, 19, 0)
(53, 102, 53, 19, 0)
(53, 103, 53, 19, 0)
(53, 104, 53, 19, 0)
(53, 105, 53, 19, 0)
(53, 106, 53, 19, 0)
(53, 107, 53, 19, 0)
(53, 108, 53, 19, 0)
(53, 109, 53, 19, 0)
(53, 110, 53, 19, 0)
(53, 111, 53, 19, 0)
(53, 112, 53, 19, 0)
(53, 113, 53, 19, 0)
(53, 114, 53, 19, 0)
(53, 115, 53, 19, 0)
(53, 116, 53, 19, 0)
(53, 117, 53, 19, 0)
(53, 118, 53, 19, 0)
(53, 119, 53, 19, 0)
(53, 120, 53, 19, 0)
(53, 121, 53, 19, 0)
(53, 122, 53, 19, 0)
(53, 123, 53, 19, 0)
(53, 124, 53, 19, 0)
(53, 125, 53, 19, 0)
(53, 126, 53, 19, 0)
(53, 127, 53, 19, 0)
(53, 128, 53, 19, 0)
(53, 129, 53, 19, 0)
(53, 130, 53, 19, 0)
(53, 131, 53, 19, 0)
(53, 132, 53, 19, 0)
(53, 133, 53, 19, 0)
(53, 134, 53, 19, 0)
(53, 135, 53, 19, 0)
(53, 136, 53, 19, 0)
(53, 137, 53, 19, 0)
(53, 138, 53, 19, 0)
(53, 139, 53, 19, 0)
(53, 140, 53, 19, 0)
(53, 141, 53, 19, 0)
(53, 142, 53, 19, 0)
(53, 143, 53, 19,

(61, 100, 61, 42, 0)
(61, 101, 61, 42, 0)
(61, 102, 61, 42, 0)
(61, 103, 61, 42, 0)
(61, 104, 61, 42, 0)
(61, 105, 61, 42, 0)
(61, 106, 61, 42, 0)
(61, 107, 61, 42, 0)
(61, 108, 61, 42, 0)
(61, 109, 61, 42, 0)
(61, 110, 61, 42, 0)
(61, 111, 61, 42, 0)
(61, 112, 61, 42, 0)
(61, 113, 61, 42, 0)
(61, 114, 61, 42, 0)
(61, 115, 61, 42, 0)
(61, 116, 61, 42, 0)
(61, 117, 61, 42, 0)
(61, 118, 61, 42, 0)
(61, 119, 61, 42, 0)
(61, 120, 61, 42, 0)
(61, 121, 61, 42, 0)
(61, 122, 61, 42, 0)
(61, 123, 61, 42, 0)
(61, 124, 61, 42, 0)
(61, 125, 61, 42, 0)
(61, 126, 61, 42, 0)
(61, 127, 61, 42, 0)
(61, 128, 61, 42, 0)
(61, 129, 61, 42, 0)
(61, 130, 61, 42, 0)
(61, 131, 61, 42, 0)
(61, 132, 61, 42, 0)
(61, 133, 61, 42, 0)
(61, 134, 61, 42, 0)
(61, 135, 61, 42, 0)
(61, 136, 61, 42, 0)
(61, 137, 61, 42, 0)
(61, 138, 61, 42, 0)
(61, 139, 61, 42, 0)
(61, 140, 61, 42, 0)
(61, 141, 61, 42, 0)
(61, 142, 61, 42, 0)
(61, 143, 61, 42, 0)
(61, 144, 61, 42, 0)
(61, 145, 61, 42, 0)
(61, 146, 61, 42, 0)
(61, 147, 61,

(71, 106, 71, 25, 0)
(71, 107, 71, 25, 0)
(71, 108, 71, 25, 0)
(71, 109, 71, 25, 0)
(71, 110, 71, 25, 0)
(71, 111, 71, 25, 0)
(71, 112, 71, 25, 0)
(71, 113, 71, 25, 0)
(71, 114, 71, 25, 0)
(71, 115, 71, 25, 0)
(71, 116, 71, 25, 0)
(71, 117, 71, 25, 0)
(71, 118, 71, 25, 0)
(71, 119, 71, 25, 0)
(71, 120, 71, 25, 0)
(71, 121, 71, 25, 0)
(71, 122, 71, 25, 0)
(71, 123, 71, 25, 0)
(71, 124, 71, 25, 0)
(71, 125, 71, 25, 0)
(71, 126, 71, 25, 0)
(71, 127, 71, 25, 0)
(71, 128, 71, 25, 0)
(71, 129, 71, 25, 0)
(71, 130, 71, 25, 0)
(71, 131, 71, 25, 0)
(71, 132, 71, 25, 0)
(71, 133, 71, 25, 0)
(71, 134, 71, 25, 0)
(71, 135, 71, 25, 0)
(71, 136, 71, 25, 0)
(71, 137, 71, 25, 0)
(71, 138, 71, 25, 0)
(71, 139, 71, 25, 0)
(71, 140, 71, 25, 0)
(71, 141, 71, 25, 0)
(71, 142, 71, 25, 0)
(71, 143, 71, 25, 0)
(71, 144, 71, 25, 0)
(71, 145, 71, 25, 0)
(71, 146, 71, 25, 0)
(71, 147, 71, 25, 0)
(71, 148, 71, 25, 0)
(71, 149, 71, 25, 0)
(71, 150, 71, 25, 0)
(71, 151, 71, 25, 0)
(71, 152, 71, 25, 0)
(71, 153, 71,

(79, 109, 79, 54, 0)
(79, 110, 79, 54, 0)
(79, 111, 79, 54, 0)
(79, 112, 79, 54, 0)
(79, 113, 79, 54, 0)
(79, 114, 79, 54, 0)
(79, 115, 79, 54, 0)
(79, 116, 79, 54, 0)
(79, 117, 79, 54, 0)
(79, 118, 79, 54, 0)
(79, 119, 79, 54, 0)
(79, 120, 79, 54, 0)
(79, 121, 79, 54, 0)
(79, 122, 79, 54, 0)
(79, 123, 79, 54, 0)
(79, 124, 79, 54, 0)
(79, 125, 79, 54, 0)
(79, 126, 79, 54, 0)
(79, 127, 79, 54, 0)
(79, 128, 79, 54, 0)
(79, 129, 79, 54, 0)
(79, 130, 79, 54, 0)
(79, 131, 79, 54, 0)
(79, 132, 79, 54, 0)
(79, 133, 79, 54, 0)
(79, 134, 79, 54, 0)
(79, 135, 79, 54, 0)
(79, 136, 79, 54, 0)
(79, 137, 79, 54, 0)
(79, 138, 79, 54, 0)
(79, 139, 79, 54, 0)
(79, 140, 79, 54, 0)
(79, 141, 79, 54, 0)
(79, 142, 79, 54, 0)
(79, 143, 79, 54, 0)
(79, 144, 79, 54, 0)
(79, 145, 79, 54, 0)
(79, 146, 79, 54, 0)
(79, 147, 79, 54, 0)
(79, 148, 79, 54, 0)
(79, 149, 79, 54, 0)
(79, 150, 79, 54, 0)
(79, 151, 79, 54, 0)
(79, 152, 79, 54, 0)
(79, 153, 79, 54, 0)
(79, 154, 79, 54, 0)
(79, 155, 79, 54, 0)
(79, 156, 79,

(89, 114, 89, 31, 0)
(89, 115, 89, 31, 0)
(89, 116, 89, 31, 0)
(89, 117, 89, 31, 0)
(89, 118, 89, 31, 0)
(89, 119, 89, 31, 0)
(89, 120, 89, 31, 0)
(89, 121, 89, 31, 0)
(89, 122, 89, 31, 0)
(89, 123, 89, 31, 0)
(89, 124, 89, 31, 0)
(89, 125, 89, 31, 0)
(89, 126, 89, 31, 0)
(89, 127, 89, 31, 0)
(89, 128, 89, 31, 0)
(89, 129, 89, 31, 0)
(89, 130, 89, 31, 0)
(89, 131, 89, 31, 0)
(89, 132, 89, 31, 0)
(89, 133, 89, 31, 0)
(89, 134, 89, 31, 0)
(89, 135, 89, 31, 0)
(89, 136, 89, 31, 0)
(89, 137, 89, 31, 0)
(89, 138, 89, 31, 0)
(89, 139, 89, 31, 0)
(89, 140, 89, 31, 0)
(89, 141, 89, 31, 0)
(89, 142, 89, 31, 0)
(89, 143, 89, 31, 0)
(89, 144, 89, 31, 0)
(89, 145, 89, 31, 0)
(89, 146, 89, 31, 0)
(89, 147, 89, 31, 0)
(89, 148, 89, 31, 0)
(89, 149, 89, 31, 0)
(89, 150, 89, 31, 0)
(89, 151, 89, 31, 0)
(89, 152, 89, 31, 0)
(89, 153, 89, 31, 0)
(89, 154, 89, 31, 0)
(89, 155, 89, 31, 0)
(89, 156, 89, 31, 0)
(89, 157, 89, 31, 0)
(89, 158, 89, 31, 0)
(89, 159, 89, 31, 0)
(89, 160, 89, 31, 0)
(89, 161, 89,

In [11]:
# conjecture 5.3
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
triples=[]
for n in [3..100]:
    p=prime(n)
   
    if (p>len(s))==False:
        power=s[p-1][0] # = p.
        poly=D(s[p-1][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..200]:
            m=p^j
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            core=polydm/p^o
            mcp=mod(core,p)
            test=o-(j*p-2)
            tupl=(p,j,power,mcp,test)
            tripl=(p,power,mcp)
            triples=triples+[tripl]
            data=data+[test]
            print(tupl)
print(data);print()
ult=uniteList(triples)
lt=len(ult);
for a in [0..lt-1]:
    print(ult[a])

(5, 1, 5, 3, 0)
(5, 2, 5, 3, 0)
(5, 3, 5, 3, 0)
(5, 4, 5, 3, 0)
(5, 5, 5, 3, 0)
(5, 6, 5, 3, 0)
(5, 7, 5, 3, 0)
(5, 8, 5, 3, 0)
(5, 9, 5, 3, 0)
(5, 10, 5, 3, 0)
(5, 11, 5, 3, 0)
(5, 12, 5, 3, 0)
(5, 13, 5, 3, 0)
(5, 14, 5, 3, 0)
(5, 15, 5, 3, 0)
(5, 16, 5, 3, 0)
(5, 17, 5, 3, 0)
(5, 18, 5, 3, 0)
(5, 19, 5, 3, 0)
(5, 20, 5, 3, 0)
(5, 21, 5, 3, 0)
(5, 22, 5, 3, 0)
(5, 23, 5, 3, 0)
(5, 24, 5, 3, 0)
(5, 25, 5, 3, 0)
(5, 26, 5, 3, 0)
(5, 27, 5, 3, 0)
(5, 28, 5, 3, 0)
(5, 29, 5, 3, 0)
(5, 30, 5, 3, 0)
(5, 31, 5, 3, 0)
(5, 32, 5, 3, 0)
(5, 33, 5, 3, 0)
(5, 34, 5, 3, 0)
(5, 35, 5, 3, 0)
(5, 36, 5, 3, 0)
(5, 37, 5, 3, 0)
(5, 38, 5, 3, 0)
(5, 39, 5, 3, 0)
(5, 40, 5, 3, 0)
(5, 41, 5, 3, 0)
(5, 42, 5, 3, 0)
(5, 43, 5, 3, 0)
(5, 44, 5, 3, 0)
(5, 45, 5, 3, 0)
(5, 46, 5, 3, 0)
(5, 47, 5, 3, 0)
(5, 48, 5, 3, 0)
(5, 49, 5, 3, 0)
(5, 50, 5, 3, 0)
(5, 51, 5, 3, 0)
(5, 52, 5, 3, 0)
(5, 53, 5, 3, 0)
(5, 54, 5, 3, 0)
(5, 55, 5, 3, 0)
(5, 56, 5, 3, 0)
(5, 57, 5, 3, 0)
(5, 58, 5, 3, 0)
(5, 59, 5, 3, 0)
(5, 60

(11, 72, 11, 5, 0)
(11, 73, 11, 5, 0)
(11, 74, 11, 5, 0)
(11, 75, 11, 5, 0)
(11, 76, 11, 5, 0)
(11, 77, 11, 5, 0)
(11, 78, 11, 5, 0)
(11, 79, 11, 5, 0)
(11, 80, 11, 5, 0)
(11, 81, 11, 5, 0)
(11, 82, 11, 5, 0)
(11, 83, 11, 5, 0)
(11, 84, 11, 5, 0)
(11, 85, 11, 5, 0)
(11, 86, 11, 5, 0)
(11, 87, 11, 5, 0)
(11, 88, 11, 5, 0)
(11, 89, 11, 5, 0)
(11, 90, 11, 5, 0)
(11, 91, 11, 5, 0)
(11, 92, 11, 5, 0)
(11, 93, 11, 5, 0)
(11, 94, 11, 5, 0)
(11, 95, 11, 5, 0)
(11, 96, 11, 5, 0)
(11, 97, 11, 5, 0)
(11, 98, 11, 5, 0)
(11, 99, 11, 5, 0)
(11, 100, 11, 5, 0)
(11, 101, 11, 5, 0)
(11, 102, 11, 5, 0)
(11, 103, 11, 5, 0)
(11, 104, 11, 5, 0)
(11, 105, 11, 5, 0)
(11, 106, 11, 5, 0)
(11, 107, 11, 5, 0)
(11, 108, 11, 5, 0)
(11, 109, 11, 5, 0)
(11, 110, 11, 5, 0)
(11, 111, 11, 5, 0)
(11, 112, 11, 5, 0)
(11, 113, 11, 5, 0)
(11, 114, 11, 5, 0)
(11, 115, 11, 5, 0)
(11, 116, 11, 5, 0)
(11, 117, 11, 5, 0)
(11, 118, 11, 5, 0)
(11, 119, 11, 5, 0)
(11, 120, 11, 5, 0)
(11, 121, 11, 5, 0)
(11, 122, 11, 5, 0)
(11, 123

(17, 109, 17, 7, 0)
(17, 110, 17, 7, 0)
(17, 111, 17, 7, 0)
(17, 112, 17, 7, 0)
(17, 113, 17, 7, 0)
(17, 114, 17, 7, 0)
(17, 115, 17, 7, 0)
(17, 116, 17, 7, 0)
(17, 117, 17, 7, 0)
(17, 118, 17, 7, 0)
(17, 119, 17, 7, 0)
(17, 120, 17, 7, 0)
(17, 121, 17, 7, 0)
(17, 122, 17, 7, 0)
(17, 123, 17, 7, 0)
(17, 124, 17, 7, 0)
(17, 125, 17, 7, 0)
(17, 126, 17, 7, 0)
(17, 127, 17, 7, 0)
(17, 128, 17, 7, 0)
(17, 129, 17, 7, 0)
(17, 130, 17, 7, 0)
(17, 131, 17, 7, 0)
(17, 132, 17, 7, 0)
(17, 133, 17, 7, 0)
(17, 134, 17, 7, 0)
(17, 135, 17, 7, 0)
(17, 136, 17, 7, 0)
(17, 137, 17, 7, 0)
(17, 138, 17, 7, 0)
(17, 139, 17, 7, 0)
(17, 140, 17, 7, 0)
(17, 141, 17, 7, 0)
(17, 142, 17, 7, 0)
(17, 143, 17, 7, 0)
(17, 144, 17, 7, 0)
(17, 145, 17, 7, 0)
(17, 146, 17, 7, 0)
(17, 147, 17, 7, 0)
(17, 148, 17, 7, 0)
(17, 149, 17, 7, 0)
(17, 150, 17, 7, 0)
(17, 151, 17, 7, 0)
(17, 152, 17, 7, 0)
(17, 153, 17, 7, 0)
(17, 154, 17, 7, 0)
(17, 155, 17, 7, 0)
(17, 156, 17, 7, 0)
(17, 157, 17, 7, 0)
(17, 158, 17, 7, 0)


(23, 126, 23, 9, 0)
(23, 127, 23, 9, 0)
(23, 128, 23, 9, 0)
(23, 129, 23, 9, 0)
(23, 130, 23, 9, 0)
(23, 131, 23, 9, 0)
(23, 132, 23, 9, 0)
(23, 133, 23, 9, 0)
(23, 134, 23, 9, 0)
(23, 135, 23, 9, 0)
(23, 136, 23, 9, 0)
(23, 137, 23, 9, 0)
(23, 138, 23, 9, 0)
(23, 139, 23, 9, 0)
(23, 140, 23, 9, 0)
(23, 141, 23, 9, 0)
(23, 142, 23, 9, 0)
(23, 143, 23, 9, 0)
(23, 144, 23, 9, 0)
(23, 145, 23, 9, 0)
(23, 146, 23, 9, 0)
(23, 147, 23, 9, 0)
(23, 148, 23, 9, 0)
(23, 149, 23, 9, 0)
(23, 150, 23, 9, 0)
(23, 151, 23, 9, 0)
(23, 152, 23, 9, 0)
(23, 153, 23, 9, 0)
(23, 154, 23, 9, 0)
(23, 155, 23, 9, 0)
(23, 156, 23, 9, 0)
(23, 157, 23, 9, 0)
(23, 158, 23, 9, 0)
(23, 159, 23, 9, 0)
(23, 160, 23, 9, 0)
(23, 161, 23, 9, 0)
(23, 162, 23, 9, 0)
(23, 163, 23, 9, 0)
(23, 164, 23, 9, 0)
(23, 165, 23, 9, 0)
(23, 166, 23, 9, 0)
(23, 167, 23, 9, 0)
(23, 168, 23, 9, 0)
(23, 169, 23, 9, 0)
(23, 170, 23, 9, 0)
(23, 171, 23, 9, 0)
(23, 172, 23, 9, 0)
(23, 173, 23, 9, 0)
(23, 174, 23, 9, 0)
(23, 175, 23, 9, 0)


(31, 155, 31, 22, 0)
(31, 156, 31, 22, 0)
(31, 157, 31, 22, 0)
(31, 158, 31, 22, 0)
(31, 159, 31, 22, 0)
(31, 160, 31, 22, 0)
(31, 161, 31, 22, 0)
(31, 162, 31, 22, 0)
(31, 163, 31, 22, 0)
(31, 164, 31, 22, 0)
(31, 165, 31, 22, 0)
(31, 166, 31, 22, 0)
(31, 167, 31, 22, 0)
(31, 168, 31, 22, 0)
(31, 169, 31, 22, 0)
(31, 170, 31, 22, 0)
(31, 171, 31, 22, 0)
(31, 172, 31, 22, 0)
(31, 173, 31, 22, 0)
(31, 174, 31, 22, 0)
(31, 175, 31, 22, 0)
(31, 176, 31, 22, 0)
(31, 177, 31, 22, 0)
(31, 178, 31, 22, 0)
(31, 179, 31, 22, 0)
(31, 180, 31, 22, 0)
(31, 181, 31, 22, 0)
(31, 182, 31, 22, 0)
(31, 183, 31, 22, 0)
(31, 184, 31, 22, 0)
(31, 185, 31, 22, 0)
(31, 186, 31, 22, 0)
(31, 187, 31, 22, 0)
(31, 188, 31, 22, 0)
(31, 189, 31, 22, 0)
(31, 190, 31, 22, 0)
(31, 191, 31, 22, 0)
(31, 192, 31, 22, 0)
(31, 193, 31, 22, 0)
(31, 194, 31, 22, 0)
(31, 195, 31, 22, 0)
(31, 196, 31, 22, 0)
(31, 197, 31, 22, 0)
(31, 198, 31, 22, 0)
(31, 199, 31, 22, 0)
(31, 200, 31, 22, 0)
(37, 1, 37, 26, 0)
(37, 2, 37, 26,

...

(59, 176, 59, 21, 0)
(59, 177, 59, 21, 0)
(59, 178, 59, 21, 0)
(59, 179, 59, 21, 0)
(59, 180, 59, 21, 0)
(59, 181, 59, 21, 0)
(59, 182, 59, 21, 0)
(59, 183, 59, 21, 0)
(59, 184, 59, 21, 0)
(59, 185, 59, 21, 0)
(59, 186, 59, 21, 0)
(59, 187, 59, 21, 0)
(59, 188, 59, 21, 0)
(59, 189, 59, 21, 0)
(59, 190, 59, 21, 0)
(59, 191, 59, 21, 0)
(59, 192, 59, 21, 0)
(59, 193, 59, 21, 0)
(59, 194, 59, 21, 0)
(59, 195, 59, 21, 0)
(59, 196, 59, 21, 0)
(59, 197, 59, 21, 0)
(59, 198, 59, 21, 0)
(59, 199, 59, 21, 0)
(59, 200, 59, 21, 0)
(61, 1, 61, 42, 0)
(61, 2, 61, 42, 0)
(61, 3, 61, 42, 0)
(61, 4, 61, 42, 0)
(61, 5, 61, 42, 0)
(61, 6, 61, 42, 0)
(61, 7, 61, 42, 0)
(61, 8, 61, 42, 0)
(61, 9, 61, 42, 0)
(61, 10, 61, 42, 0)
(61, 11, 61, 42, 0)
(61, 12, 61, 42, 0)
(61, 13, 61, 42, 0)
(61, 14, 61, 42, 0)
(61, 15, 61, 42, 0)
(61, 16, 61, 42, 0)
(61, 17, 61, 42, 0)
(61, 18, 61, 42, 0)
(61, 19, 61, 42, 0)
(61, 20, 61, 42, 0)
(61, 21, 61, 42, 0)
(61, 22, 61, 42, 0)
(61, 23, 61, 42, 0)
(61, 24, 61, 42, 0)
(61,

(67, 178, 67, 46, 0)
(67, 179, 67, 46, 0)
(67, 180, 67, 46, 0)
(67, 181, 67, 46, 0)
(67, 182, 67, 46, 0)
(67, 183, 67, 46, 0)
(67, 184, 67, 46, 0)
(67, 185, 67, 46, 0)
(67, 186, 67, 46, 0)
(67, 187, 67, 46, 0)
(67, 188, 67, 46, 0)
(67, 189, 67, 46, 0)
(67, 190, 67, 46, 0)
(67, 191, 67, 46, 0)
(67, 192, 67, 46, 0)
(67, 193, 67, 46, 0)
(67, 194, 67, 46, 0)
(67, 195, 67, 46, 0)
(67, 196, 67, 46, 0)
(67, 197, 67, 46, 0)
(67, 198, 67, 46, 0)
(67, 199, 67, 46, 0)
(67, 200, 67, 46, 0)
(71, 1, 71, 25, 0)
(71, 2, 71, 25, 0)
(71, 3, 71, 25, 0)
(71, 4, 71, 25, 0)
(71, 5, 71, 25, 0)
(71, 6, 71, 25, 0)
(71, 7, 71, 25, 0)
(71, 8, 71, 25, 0)
(71, 9, 71, 25, 0)
(71, 10, 71, 25, 0)
(71, 11, 71, 25, 0)
(71, 12, 71, 25, 0)
(71, 13, 71, 25, 0)
(71, 14, 71, 25, 0)
(71, 15, 71, 25, 0)
(71, 16, 71, 25, 0)
(71, 17, 71, 25, 0)
(71, 18, 71, 25, 0)
(71, 19, 71, 25, 0)
(71, 20, 71, 25, 0)
(71, 21, 71, 25, 0)
(71, 22, 71, 25, 0)
(71, 23, 71, 25, 0)
(71, 24, 71, 25, 0)
(71, 25, 71, 25, 0)
(71, 26, 71, 25, 0)
(71, 2

(73, 183, 73, 50, 0)
(73, 184, 73, 50, 0)
(73, 185, 73, 50, 0)
(73, 186, 73, 50, 0)
(73, 187, 73, 50, 0)
(73, 188, 73, 50, 0)
(73, 189, 73, 50, 0)
(73, 190, 73, 50, 0)
(73, 191, 73, 50, 0)
(73, 192, 73, 50, 0)
(73, 193, 73, 50, 0)
(73, 194, 73, 50, 0)
(73, 195, 73, 50, 0)
(73, 196, 73, 50, 0)
(73, 197, 73, 50, 0)
(73, 198, 73, 50, 0)
(73, 199, 73, 50, 0)
(73, 200, 73, 50, 0)
(79, 1, 79, 54, 0)
(79, 2, 79, 54, 0)
(79, 3, 79, 54, 0)
(79, 4, 79, 54, 0)
(79, 5, 79, 54, 0)
(79, 6, 79, 54, 0)
(79, 7, 79, 54, 0)
(79, 8, 79, 54, 0)
(79, 9, 79, 54, 0)
(79, 10, 79, 54, 0)
(79, 11, 79, 54, 0)
(79, 12, 79, 54, 0)
(79, 13, 79, 54, 0)
(79, 14, 79, 54, 0)
(79, 15, 79, 54, 0)
(79, 16, 79, 54, 0)
(79, 17, 79, 54, 0)
(79, 18, 79, 54, 0)
(79, 19, 79, 54, 0)
(79, 20, 79, 54, 0)
(79, 21, 79, 54, 0)
(79, 22, 79, 54, 0)
(79, 23, 79, 54, 0)
(79, 24, 79, 54, 0)
(79, 25, 79, 54, 0)
(79, 26, 79, 54, 0)
(79, 27, 79, 54, 0)
(79, 28, 79, 54, 0)
(79, 29, 79, 54, 0)
(79, 30, 79, 54, 0)
(79, 31, 79, 54, 0)
(79, 32, 79

(83, 184, 83, 29, 0)
(83, 185, 83, 29, 0)
(83, 186, 83, 29, 0)
(83, 187, 83, 29, 0)
(83, 188, 83, 29, 0)
(83, 189, 83, 29, 0)
(83, 190, 83, 29, 0)
(83, 191, 83, 29, 0)
(83, 192, 83, 29, 0)
(83, 193, 83, 29, 0)
(83, 194, 83, 29, 0)
(83, 195, 83, 29, 0)
(83, 196, 83, 29, 0)
(83, 197, 83, 29, 0)
(83, 198, 83, 29, 0)
(83, 199, 83, 29, 0)
(83, 200, 83, 29, 0)
(89, 1, 89, 31, 0)
(89, 2, 89, 31, 0)
(89, 3, 89, 31, 0)
(89, 4, 89, 31, 0)
(89, 5, 89, 31, 0)
(89, 6, 89, 31, 0)
(89, 7, 89, 31, 0)
(89, 8, 89, 31, 0)
(89, 9, 89, 31, 0)
(89, 10, 89, 31, 0)
(89, 11, 89, 31, 0)
(89, 12, 89, 31, 0)
(89, 13, 89, 31, 0)
(89, 14, 89, 31, 0)
(89, 15, 89, 31, 0)
(89, 16, 89, 31, 0)
(89, 17, 89, 31, 0)
(89, 18, 89, 31, 0)
(89, 19, 89, 31, 0)
(89, 20, 89, 31, 0)
(89, 21, 89, 31, 0)
(89, 22, 89, 31, 0)
(89, 23, 89, 31, 0)
(89, 24, 89, 31, 0)
(89, 25, 89, 31, 0)
(89, 26, 89, 31, 0)
(89, 27, 89, 31, 0)
(89, 28, 89, 31, 0)
(89, 29, 89, 31, 0)
(89, 30, 89, 31, 0)
(89, 31, 89, 31, 0)
(89, 32, 89, 31, 0)
(89, 33, 89,

(97, 186, 97, 66, 0)
(97, 187, 97, 66, 0)
(97, 188, 97, 66, 0)
(97, 189, 97, 66, 0)
(97, 190, 97, 66, 0)
(97, 191, 97, 66, 0)
(97, 192, 97, 66, 0)
(97, 193, 97, 66, 0)
(97, 194, 97, 66, 0)
(97, 195, 97, 66, 0)
(97, 196, 97, 66, 0)
(97, 197, 97, 66, 0)
(97, 198, 97, 66, 0)
(97, 199, 97, 66, 0)
(97, 200, 97, 66, 0)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [12]:
dat=[]
for a in [0..lt-1]:
    dat=dat+[ult[a][2]]
print(dat)

[3, 6, 5, 10, 7, 14, 9, 11, 22, 26, 15, 30, 17, 19, 21, 42, 46, 25, 50, 54, 29, 31, 66]


In [13]:
for a in [0..lt-1]:
    print(ult[a])

(5, 5, 3)
(7, 7, 6)
(11, 11, 5)
(13, 13, 10)
(17, 17, 7)
(19, 19, 14)
(23, 23, 9)
(29, 29, 11)
(31, 31, 22)
(37, 37, 26)
(41, 41, 15)
(43, 43, 30)
(47, 47, 17)
(53, 53, 19)
(59, 59, 21)
(61, 61, 42)
(67, 67, 46)
(71, 71, 25)
(73, 73, 50)
(79, 79, 54)
(83, 83, 29)
(89, 89, 31)
(97, 97, 66)
